In [1]:
from New2048 import Game2048 as _2048
import numpy as np
import New2048

In [2]:
game = _2048(4)
game.start_game
game.playAdversaryAction()
#game.playRandUser()
print(game)
print(game.board_value())

+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
|      |      |      |  2   |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
|      |      |      |      |
+------+------+------+------+
4


In [3]:
#New2048.main()

# TEST 1 - Monte Carlo Tree Search

In [4]:
# Starter Code: MCTS Implementation
import time


class Tree():
    def __init__(self, *, start_grid=None, parent=None, move=None):
        if parent is None:
            self.parent = None
            self.move = None
            self.grid = start_grid
        else:
            self.parent = parent
            self.move = move
            self.grid = parent.grid.copy()
            if self.grid.play_state:
                self.grid.playUserAction(move)
            else:
                self.grid.playAdversaryAction()
        
        self.values = []
        self.n = 0
        if self.is_terminal_state:
            self.unexplored_moves = set()
        else:
            if self.grid.play_state:
                self.unexplored_moves = set(self.grid.get_user_actions())
            else:
                #print(self.grid.getAdverseryActions())
                self.unexplored_moves = set(self.grid.getAdverseryActions())
        self.children = set([])
            
    @property
    def fully_expanded(self):
        return len(self.unexplored_moves) == 0
    
    @property
    def is_terminal_state(self):
        return (self.grid.winner is not None)


def monte_carlo_tree_search(start_grid, num_iterations=1000):
    """MCTS core loop"""
    # Start by creating the root of the tree.
    root = Tree(start_grid=start_grid)
    
    # Loop through MCTS iterations.
    for _ in range(num_iterations):
        # One step of MCTS iteration
        leaf = traverse(root)
        sim_res = rollout(leaf, start_grid)
        backpropagate(leaf, sim_res)

    # When done iterating, return the 'best' child of the root node.
    return best_child(root).move

def best_child(node):
    """When done sampling, pick the child visited the most."""
    #print(node)
    return max(node.children, key=lambda child: child.n)

'''def worst_child(node):
    """ Exact opposite of the best child (Least visited node) """
    return min(node.children, key=lambda child: child.n)'''


def best_uct(node, C=5):
    """Pick the best action according to the UCB/UCT algorithm"""
    chos_val = float("-inf")
    for nub in node.children:
        temp = np.sum(nub.values)/nub.n + C * np.sqrt(np.log(node.n)/nub.n)
        if temp >chos_val:
            chos_val = temp
            chose = nub
    try:
        return chose
    except:
        print(node.grid)
        return chose


def traverse(node):
    # If fully explored, pick one of the children
    while node.fully_expanded and not node.is_terminal_state:
        node = best_uct(node)

    # If the node is terminal, return it
    if node.is_terminal_state:
        return node
    
    # If the node is not terminal:
    # 1. pick a new move from 'unexplored_moves'
    move = node.unexplored_moves.pop()
    # 2. create a new child
    new_child = Tree(parent=node, move=move)
    # 3. add that child to the list of children
    node.children.add(new_child)
    # 4. return that new child
    return new_child


def rollout(node, start_grid):
    sample = node.grid.copy()
    winner = sample.play_random_moves_until_done()
    '''if winner == 0:
        return 0
    elif winner == start_grid.play_state:
        return 1
    else:
        return -1'''
    return sample.board_value()/(2048 ** 2)


def backpropagate(node, simulation_result):
    """Update the node and its parent (via recursion)."""
    if node is None:
        return
    node.n += 1
    node.values.append(simulation_result)
    backpropagate(node.parent, simulation_result)
    

In [8]:

time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(6)
    tot_time = 0
    while board.winner == None:
        #print(board)
        if board.play_state:
            stime = time.time()
            action = monte_carlo_tree_search(board, 250)
            tot_time += stime
            board.playUserAction(action)
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True
    if board.winner == 1:
        wins += 1
        win_arr.append(1)
    else:
        win_arr.append(0)
    time_arr.append(tot_time)
    print(i)
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)
    

0
+------+------+------+------+------+------+
|      |      | 256  |      |      |      |
+------+------+------+------+------+------+
|  2   |  4   |  16  |      |      |      |
+------+------+------+------+------+------+
|  64  | 128  |  64  |  2   | 512  |      |
+------+------+------+------+------+------+
|  2   |  2   |  4   |  32  |      |      |
+------+------+------+------+------+------+
|  32  |  8   | 256  |  2   | 512  |  2   |
+------+------+------+------+------+------+
|  4   | 512  |  4   |  4   | 2048 |  4   |
+------+------+------+------+------+------+
1
5138872.0
125979953705.86163
1
+------+------+------+------+------+------+
|  32  |  16  |  32  |  8   |      |      |
+------+------+------+------+------+------+
| 256  |  32  |  8   | 1024 |  8   |  2   |
+------+------+------+------+------+------+
|  4   | 2048 |      |      |      |      |
+------+------+------+------+------+------+
|  2   |  4   |      |      |      |      |
+------+------+------+------+------+-----

14
+------+------+------+------+------+------+
|      |      | 1024 |  8   |  8   |  2   |
+------+------+------+------+------+------+
|      |  8   |  4   |  16  | 128  |  64  |
+------+------+------+------+------+------+
|  2   |      |  2   | 1024 |  4   |  16  |
+------+------+------+------+------+------+
|      |      |      |  32  |  32  |  8   |
+------+------+------+------+------+------+
|      |      | 1024 |      | 1024 | 2048 |
+------+------+------+------+------+------+
|      |      |  2   |      |  2   |  4   |
+------+------+------+------+------+------+
1
8411972.0
151516446635.44638
15
+------+------+------+------+------+------+
|      |  2   |  4   |  2   | 512  | 2048 |
+------+------+------+------+------+------+
| 128  |  64  |  8   |  2   |  64  | 256  |
+------+------+------+------+------+------+
|  16  |  8   |  32  |  4   | 128  |  2   |
+------+------+------+------+------+------+
|      |  2   |  32  | 1024 |  8   |  4   |
+------+------+------+------+------+---

28
+------+------+------+------+------+------+
|  4   |  32  |  32  |  4   |  2   | 2048 |
+------+------+------+------+------+------+
|  8   | 128  |  64  |  16  |  16  | 512  |
+------+------+------+------+------+------+
| 512  | 1024 |  16  |  2   |  4   |      |
+------+------+------+------+------+------+
|  2   |  64  |  4   | 128  |      | 512  |
+------+------+------+------+------+------+
|  4   |  4   |  2   |  4   |      |      |
+------+------+------+------+------+------+
|      |      |      |  2   |      |      |
+------+------+------+------+------+------+
1
6073284.0
240042935607.8204
29
+------+------+------+------+------+------+
|      |  2   |  4   |  16  |  2   |  8   |
+------+------+------+------+------+------+
|      |  8   |  16  | 128  |  4   |  16  |
+------+------+------+------+------+------+
|  4   |  32  |  2   |  32  |  8   |  4   |
+------+------+------+------+------+------+
|      |      |  2   |  8   |  32  |  4   |
+------+------+------+------+------+----

42
+------+------+------+------+------+------+
|  4   |  2   |  32  |  4   |  32  |  64  |
+------+------+------+------+------+------+
|  16  |  64  | 128  |  2   |  4   |      |
+------+------+------+------+------+------+
|  8   | 2048 |  32  |  16  |      |      |
+------+------+------+------+------+------+
|  64  |  2   |      |      |      |      |
+------+------+------+------+------+------+
|  4   |  32  |  16  |      |  16  |      |
+------+------+------+------+------+------+
|  2   |  8   |  2   |      |  2   |      |
+------+------+------+------+------+------+
1
4228312.0
328569441831.1194
43
+------+------+------+------+------+------+
|  2   |  16  |  8   |  8   |  16  |  8   |
+------+------+------+------+------+------+
|  4   | 256  |  64  |  2   |  8   |  2   |
+------+------+------+------+------+------+
|      |  32  | 256  |  8   |  32  |  8   |
+------+------+------+------+------+------+
|      |  2   | 2048 |  4   | 512  |  4   |
+------+------+------+------+------+----

56
+------+------+------+------+------+------+
|  4   |  8   |  4   |  32  | 2048 |  4   |
+------+------+------+------+------+------+
|  32  | 512  |  64  | 512  | 512  |  2   |
+------+------+------+------+------+------+
| 128  |  16  |  16  |  4   |      |  4   |
+------+------+------+------+------+------+
|  4   |  2   |  64  |  4   | 512  |      |
+------+------+------+------+------+------+
|  2   |      |  2   |      |      |      |
+------+------+------+------+------+------+
|      |      |      |      |      |      |
+------+------+------+------+------+------+
1
5270208.0
141301902850.28064
57
+------+------+------+------+------+------+
|  2   |  8   |  2   |  64  |  8   |  4   |
+------+------+------+------+------+------+
|  8   |  16  | 2048 |  64  |      |      |
+------+------+------+------+------+------+
|  4   |  8   |  64  |  8   |      |      |
+------+------+------+------+------+------+
|  4   |  2   | 1024 |  2   |  4   |      |
+------+------+------+------+------+---

70
+------+------+------+------+------+------+
|  32  |  2   |  8   |      |      |      |
+------+------+------+------+------+------+
|  4   |  16  |  32  |      |      |      |
+------+------+------+------+------+------+
|  8   |  2   |  64  |      |      |  2   |
+------+------+------+------+------+------+
|  2   | 256  | 128  |  16  |      |  4   |
+------+------+------+------+------+------+
|  16  | 512  |  4   |  8   |      |  2   |
+------+------+------+------+------+------+
|  4   |  2   |  2   |  2   | 2048 |  8   |
+------+------+------+------+------+------+
1
4545632.0
182160300001.0401
71
+------+------+------+------+------+------+
|      |      |      |      |      |      |
+------+------+------+------+------+------+
|      |      |      |  2   |  4   |  8   |
+------+------+------+------+------+------+
|      | 512  |      |  8   | 256  |  16  |
+------+------+------+------+------+------+
|  4   |      |  4   |  32  |  2   |  4   |
+------+------+------+------+------+----

84
+------+------+------+------+------+------+
|      |      |      |      |      |  4   |
+------+------+------+------+------+------+
|      |      |      |      |  16  | 256  |
+------+------+------+------+------+------+
|      |  4   | 128  |  64  | 128  |  2   |
+------+------+------+------+------+------+
|      |  4   |  8   |  32  |  8   | 256  |
+------+------+------+------+------+------+
|      |  4   |  64  | 128  |  16  | 512  |
+------+------+------+------+------+------+
|      |  4   | 2048 |  8   |  2   |  16  |
+------+------+------+------+------+------+
1
4646936.0
100443538335.3147
85
+------+------+------+------+------+------+
|      |      |      |      |      |      |
+------+------+------+------+------+------+
| 128  |      |      |      |  8   |  4   |
+------+------+------+------+------+------+
|      |      |      | 256  |  16  |  8   |
+------+------+------+------+------+------+
| 128  |      |  16  |  2   | 256  |  2   |
+------+------+------+------+------+----

98
+------+------+------+------+------+------+
|  2   |  4   |  2   |      |      |      |
+------+------+------+------+------+------+
|  16  |  32  |  4   |      |      |      |
+------+------+------+------+------+------+
|  2   |  8   |  32  |  2   |  16  |      |
+------+------+------+------+------+------+
|  2   | 2048 |  2   |  16  |  2   |      |
+------+------+------+------+------+------+
|  4   |  16  | 1024 |  8   |      |      |
+------+------+------+------+------+------+
|  4   | 512  |  32  | 256  |  2   | 128  |
+------+------+------+------+------+------+
1
5591264.0
91931382044.31923
99
+------+------+------+------+------+------+
|      |      |      |      |  2   |      |
+------+------+------+------+------+------+
|      |      |      |  4   |  4   |  4   |
+------+------+------+------+------+------+
|      |      |      |  2   |  16  |  8   |
+------+------+------+------+------+------+
| 2048 |      | 128  |  4   |  64  |  32  |
+------+------+------+------+------+----

In [9]:
f = open("res3.csv", 'w')

for i in range(len(time_arr)):
    f.write(str(time_arr[i]) + ", " + str(win_arr[i]) + ",\n")
f.close()

# TEST 2 - n-Step MCTS

In [5]:

def mcts_with_n_step(start_grid, n_step, num_iterations = 1000):
    
    # Start by creating the root of the tree.
    root = Tree(start_grid=start_grid)
    
    # Loop through MCTS iterations.
    for _ in range(num_iterations):
        # One step of MCTS iteration
        leaf = traverse(root)
        sim_res = rollout_n_step(leaf, start_grid, n_step)
        backpropagate(leaf, sim_res)

    # When done iterating, return the 'best' child of the root node.
    return best_child(root).move


def rollout_n_step(node, start_grid, n_step):
    sample = node.grid.copy()
    winner = sample.play_random_moves_for_n(n_step)

    return sample.board_value()/(2048 ** 2)




In [ ]:
 time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(4)
    tot_time = 0
    while board.winner == None:
        #print(board)
        if board.play_state:
            stime = time.time()
            action = mcts_with_n_step(board, 100, 250)
            tot_time += stime
            board.playUserAction(action)
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True
    if board.winner == 1:
        wins += 1
        win_arr.append(1)
    else:
        win_arr.append(0)
    time_arr.append(tot_time)
    print(i)
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)

0
+------+------+------+------+
|  16  |  8   |  64  |  2   |
+------+------+------+------+
|  2   |  32  | 512  | 256  |
+------+------+------+------+
|  16  | 128  |  32  |  8   |
+------+------+------+------+
|  4   |  2   |  4   |  2   |
+------+------+------+------+
2
350896.0
209399003412.29755
1
+------+------+------+------+
|  2   |  16  |  8   |  4   |
+------+------+------+------+
|  8   |  32  |  4   |  2   |
+------+------+------+------+
|  32  |  8   |  16  |  64  |
+------+------+------+------+
|  8   | 256  |  4   |  2   |
+------+------+------+------+
2
72508.0
154921215497.19693
2
+------+------+------+------+
|  4   |  16  |  4   |  2   |
+------+------+------+------+
|  8   | 256  |  8   |  16  |
+------+------+------+------+
|  2   |  32  | 128  |  2   |
+------+------+------+------+
|  4   |  64  |  8   |  4   |
+------+------+------+------+
2
87820.0
156623647610.9081
3
+------+------+------+------+
|  2   |  4   |  32  |  2   |
+------+------+------+------+
|  64

# TEST 3 - Rand Walk

# TEST 4 - How High

In [5]:
# Starter Code: MCTS Implementation
import time


class Tree():
    def __init__(self, *, start_grid=None, parent=None, move=None):
        if parent is None:
            self.parent = None
            self.move = None
            self.grid = start_grid
        else:
            self.parent = parent
            self.move = move
            self.grid = parent.grid.copy()
            if self.grid.play_state:
                self.grid.playUserAction(move)
            else:
                self.grid.playAdversaryAction()
        
        self.values = []
        self.n = 0
        if self.is_terminal_state:
            self.unexplored_moves = set()
        else:
            if self.grid.play_state:
                self.unexplored_moves = set(self.grid.get_user_actions())
                
            else:
                #print(self.grid.getAdverseryActions())
                self.unexplored_moves = set(self.grid.getAdverseryActions())
        self.children = set([])
            
    @property
    def fully_expanded(self):
        #print(self.unexplored_moves)
        return len(self.unexplored_moves) == 0
    
    @property
    def is_terminal_state(self):
        return (self.grid.check_game_over())


def monte_carlo_tree_search(start_grid, num_iterations=1000):
    """MCTS core loop"""
    # Start by creating the root of the tree.
    root = Tree(start_grid=start_grid)
    
    # Loop through MCTS iterations.
    for _ in range(num_iterations):
        # One step of MCTS iteration
        leaf = traverse(root)
        #print(leaf.grid)
        sim_res = rollout_o(leaf, start_grid)
        #print(sim_res)
        backpropagate(leaf, sim_res)
        #print(root.children)

    # When done iterating, return the 'best' child of the root node.\
    try:
        return best_child(root).move
    except:
        #print(root.children)
        return root

def best_child(node):
    """When done sampling, pick the child visited the most."""
    #print(node)
    return max(node.children, key=lambda child: child.n)

'''def worst_child(node):
    """ Exact opposite of the best child (Least visited node) """
    return min(node.children, key=lambda child: child.n)'''


def best_uct(node, C=5):
    """Pick the best action according to the UCB/UCT algorithm"""
    chos_val = float("-inf")
    for nub in node.children:
        temp = np.sum(nub.values)/nub.n + C * np.sqrt(np.log(node.n)/nub.n)
        if temp >chos_val:
            chos_val = temp
            chose = nub
    try:
        return chose
    except:
        #print(node.grid)
        return chose


def traverse(node):
    # If fully explored, pick one of the children
    while node.fully_expanded and not node.is_terminal_state:
        node = best_uct(node)
    
    # If the node is terminal, return it
    if node.is_terminal_state:
        #print(1)
        return node
    
    # If the node is not terminal:
    # 1. pick a new move from 'unexplored_moves'
    move = node.unexplored_moves.pop()
    # 2. create a new child
    new_child = Tree(parent=node, move=move)
    # 3. add that child to the list of children
    node.children.add(new_child)
    # 4. return that new child
    return new_child


def rollout_o(node, start_grid):
    sample = node.grid.copy()
    winner = sample.play_til_over()
    return sample.board_value()/(16384 ** 2)


def backpropagate(node, simulation_result):
    """Update the node and its parent (via recursion)."""
    if node is None:
        return
    node.n += 1
    node.values.append(simulation_result)
    backpropagate(node.parent, simulation_result)
    

In [6]:
time_arr = []
win_arr = []
wins = 0
for i in range(100):
    board = _2048(5)
    tot_time = 0
    board.playAdversaryAction()
    board.play_state = True
    while board.check_game_over() == False:
        #print(board)
        if board.play_state:
            stime = time.time()
            action = monte_carlo_tree_search(board, 250)
            tot_time += stime
            board.playUserAction(action)
            board.play_state = False
        else:
            board.playAdversaryAction()
            board.play_state = True

    win_arr.append(board.get_highest_tile())
    time_arr.append(tot_time)
    print(board.get_highest_tile())
    print(board)
    print(board.winner)
    print(board.board_value())
    print(tot_time)
print("wins - " + str(wins))
print("percent - " +str(wins/100))
print(time_arr)

1048576.0
+------+------+------+------+------+
|  4   |  2   |131072|  16  |  4   |
+------+------+------+------+------+
|  32  | 128  |  8   |  32  |  8   |
+------+------+------+------+------+
|  8   |  4   |  32  | 256  |  2   |
+------+------+------+------+------+
|  4   |  8   |  4   |  64  |  4   |
+------+------+------+------+------+
|  2   |  32  |  2   |1048576|  2   |
+------+------+------+------+------+
2
1116691587700.0
457956642132.8496


KeyboardInterrupt: 